In [30]:
class Value:
    def __init__(self, data, _parents = (), _op = ''):
        self.data = data
        self._prev = set(_parents)
        self.grad = 0
        self._op = _op
        self.__update_parent_grad__ = lambda:None
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
     
    def backprop(self):
        self.__update_parent_grad__()
        for node in self._prev:
            node.backprop()
            
    def __add__(self, other):
        if type(other) is not Value:
            other = Value(other)
        
        output = Value(self.data + other.data, (self, other), '+')
        
        def __update_parent_grad__():
            self.grad += output.grad
            other.grad += output.grad
            
        output.__update_parent_grad__ = __update_parent_grad__
        return output
        
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, other):
        if type(other) is not Value:
            other = Value(other)
        output = Value(self.data * other.data, (self, other), '*')
        
        def __update_parent_grad__():
            self.grad = other.data * output.grad
            other.grad = self.data * output.grad
        output.__update_parent_grad__ = __update_parent_grad__
        return  output
    
    def __rmul__(self, other):
        return self * other

In [59]:
a = Value(1)
b = Value(2)
h = 1e-6
a_h = a + h
c = a+b
c_h = a_h+b
d = c*a

d_h = c_h*a_h

In [60]:
d.grad = 1
d.backprop()

In [61]:
c.grad

1

In [62]:
a.grad

4

In [63]:
b.grad

1

In [65]:
(d_h.data-d.data)/h

4.000000999759834

In [58]:
d

Value(data=3)